<a href="https://colab.research.google.com/github/JCaballerot/Credit-Scoring/blob/main/CreditScoring/02.%20Lab_ML_Automobile_Loan_Default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5>Automobile Loan Default Dataset
</font></h1>

---

## Introducción

En este laboratorio, aprenderá a usar python para construir un modelo de <b>credit scoring</b>.



## Objetivo de este Notebook

1. Como construir e interpretar un modelo de regresión logística.
2. Descargar y limpiar un Dataset
3. Realizar los pasos necesarios previos a la etapa de modelamiento
4. Entrenar y Testear modelo

## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Contexto</a>  
2. <a href="#item32">Descargar y preparar Dataset</a>  
3. <a href="#item33">Muestreo de datos</a>  
4. <a href="#item34">Tratamiento de variables</a>  
5. <a href="#item34">Modelamiento</a>  
6. <a href="#item34">Evaluación del modelo</a>  
7. <a href="#item34">Interpretación del modelo</a>  
8. <a href="#item34">Implementación del modelo</a>  

</font>
</div>

### 1. Contexto

Una institución financiera no bancaria (NBFI) o una compañía financiera no bancaria (NBFC) es una institución financiera que no tiene una licencia bancaria completa o no está supervisada por una agencia reguladora bancaria nacional o internacional. NBFC facilita los servicios financieros relacionados con los bancos, como inversión, agrupación de riesgos, ahorros contractuales y corretaje de mercado.

Una NBFI está luchando por marcar ganancias debido a un aumento en los incumplimientos en la categoría de préstamos para vehículos. La empresa tiene como objetivo determinar la capacidad de pago del préstamo del cliente y comprender la importancia relativa de cada parámetro que contribuye a la capacidad del prestatario para pagar el préstamo.



<b>Meta:</b>

El objetivo del problema es predecir si un cliente no cumplirá con el pago del préstamo del vehículo o no.




<img src="https://storage.googleapis.com/kaggle-datasets-images/1205706/2014650/7a50061003ce0de9839d54ff4673ccf2/dataset-cover.jpg?t=2021-07-04-09-18-01" alt="HTML5 Icon" style="width: 600px; height: 450px;">
<div style="text-align: center">¿Cómo identificaremos a los clientes morosos? </div>


<b>Descripción de datos</b>

El data frame de Automovile Loan tiene 121,856 filas y 40 columnas.


---

Se le proporciona un conjunto de datos anónimos que contiene una gran cantidad de variables. La columna <b>"Default"</b> es la variable a predecir. Es igual a uno para clientes que cayeron en mora y 0 para clientes que no.

La tarea consiste en predecir la probabilidad de que cada cliente del conjunto de prueba sea un cliente moroso.


---


<strong>Puede consultar este [link](https://www.kaggle.com/datasets/saurabhbagchi/dish-network-hackathon) para leer más sobre la fuente de datos Automobile Loan Default.</strong>


### 2. Descargar y preparar Dataset

In [15]:
# Principales librerías

# Scikit-Learn
import sklearn

# Imports comunes
import pandas as pd
import numpy as np


In [ ]:
# Cargar Data
pddf = pd.read_csv("loan_default.csv")
pddf.head()


In [ ]:
# Indentificar las variables
features = list(set(pddf.columns.tolist()) - set(['ID', 'ID_Days','Default', 'codmes']))
features

In [37]:
catergory_features = ['Accompany_Client', 'Client_Income_Type', 'Client_Education', 'Client_Marital_Status',
                      'Client_Gender', 'Loan_Contract_Type', 'Client_Housing_Type', 'Client_Occupation', 'Client_Permanent_Match_Tag', 'Client_Contact_Work_Tag', 'Type_Organization']


In [ ]:
numeric_features = list(set(features) - set(catergory_features))

# convert Series
for col in numeric_features:
  pddf[col] = pd.to_numeric(pddf[col])

### 3. Muestreo de datos

El **watchlist** es un tipo de muestra adicional que se utiliza en el entrenamiento de modelos de machine learning principalmente para monitorear el rendimiendo dado que permitirá imprimir métricas de evaluación (como el error de clasificación, el error cuadrático medio, etc.) en cada iteración del proceso de entrenamiento, ofreciendo así una traza visible del rendimiento del modelo a medida que aprende.

In [35]:
# Muestreo de data
from sklearn.model_selection import train_test_split


train, test = train_test_split(pddf,
                               stratify = pddf.Default,
                               train_size = 0.6,
                               random_state = 123)

watch, test = train_test_split(test,
                               stratify = test.Default,
                               train_size = 0.5,
                               random_state = 123)


### 4. Tratamiento de variables

**Encoding**

El encoding de variables categóricas convierte las categorías de texto en números de una manera que puede ser utilizada de manera eficiente por los algoritmos de machine learning.

In [36]:
%%capture
!pip3 install category_encoders

In [39]:
# Aplicando category encoders
from category_encoders import TargetEncoder

encoder = TargetEncoder(handle_unknown = 'infrequent_if_exist',
                        handle_missing = 'value',
                        min_samples_leaf = 30)

encoder.fit(train[catergory_features].astype('category'), train['Default'])

TargetEncoder(cols=['Accompany_Client', 'Client_Income_Type',
                    'Client_Education', 'Client_Marital_Status',
                    'Client_Gender', 'Loan_Contract_Type',
                    'Client_Housing_Type', 'Client_Occupation',
                    'Client_Permanent_Match_Tag', 'Client_Contact_Work_Tag',
                    'Type_Organization'],
              handle_unknown='infrequent_if_exist', min_samples_leaf=30)

In [40]:
# Aplicando transformaciones sobre  variables

train[[x + '_coded' for x in catergory_features]] = encoder.transform(train[catergory_features].astype('category'))
test[[ x + '_coded' for x in catergory_features]] = encoder.transform(test[catergory_features].astype('category'))
watch[[x + '_coded' for x in catergory_features]] = encoder.transform(watch[catergory_features].astype('category'))



### 6. Modelamiento


In [41]:
#Cargando librerías
import xgboost as xgb
from sklearn.metrics import *

In [72]:
# Definimos los parámetros para el Grid Search

param_grid = {'objective': ['binary:logistic'],
              'scale_pos_weight' : [11.37],
              'booster' :  ['gbtree'],
              'learning_rate': [0.01, 0.05, 0.1],
              'max_depth': [3, 5, 7],
              'colsample_bytree': [0.7, 1],
              'subsample': [0.7, 1]}


In [70]:
features =  numeric_features + [x + '_coded' for x in catergory_features]

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

# Crear clasificador
xgBoost = xgb.XGBClassifier(use_label_encoder=False, n_estimators = 500)

# Crear objeto GridSearchCV
grid_search = GridSearchCV(xgBoost,
                           param_grid,
                           scoring = make_scorer(auc),
                           cv = 3,  # Número de folds en la validación cruzada
                           verbose = 2,  # Verbosidad del output
                           n_jobs = -1  # Uso de todos los núcleos disponibles
                          )

# Realizar búsqueda de parámetros
grid_search.fit(train[features],
                train.Default,
                early_stopping_rounds = 10,
                eval_metric = "auc",
                eval_set=[(watch[features], watch.Default)],
                verbose = True)



In [ ]:
# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# Si deseas, también puedes extraer y visualizar los mejores parámetros encontrados
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")


In [46]:
%%capture
!pip install --upgrade xgboost

In [ ]:
# Entrenando el modelo final

xgBoost = xgb.XGBClassifier(use_label_encoder=False,
                            n_estimators = 500, **best_params)

xgBoost.fit(train[features],
            train.Default,
            early_stopping_rounds=10,
            eval_metric="auc",
            eval_set=[(train[features], train.Default), (watch[features], watch.Default)],
            verbose=True)


# Extraer los resultados de evaluación
results = xgBoost.evals_result()


In [ ]:
import matplotlib.pyplot as plt

epochs = len(results['validation_0']['auc'])
x_axis = range(0, epochs)

# Ajusta el tamaño
fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(x_axis, results['validation_0']['auc'], label='Train')
ax.plot(x_axis, results['validation_1']['auc'], label='Watch')

ax.set_ylim([0.65, 0.75])  # Para limitar la cantidad de epochs

ax.legend()
plt.ylabel('AUC')
plt.title('XGBoost AUC')
plt.show()

<b>Variables del Modelo</b>

In [ ]:
import matplotlib.pyplot as plt

# Obtener importancia
feature_importance = xgBoost.get_booster().get_score(importance_type='weight')

# Ordenar por importancia
sorted_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)

# Graficar
plt.barh([x[0] for x in sorted_importance[:10]], [x[1] for x in sorted_importance[:10]])
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.title('Top 10 variables por Importancia')
plt.gca().invert_yaxis()
plt.show()

### 7. Evaluación del modelo

In [80]:
from scipy.stats import ks_2samp

# Definir métricas adicionales
def gini(y_true, y_score):
    auc = roc_auc_score(y_true, y_score)
    return 2*auc - 1

def ks_statistic(y_true, y_score):
    return ks_2samp(y_score[y_true == 1], y_score[y_true == 0]).statistic

In [81]:
# predicción del modelo
train['prediction'] = xgBoost.predict_proba(train[features])[:, 1]
test['prediction']  = xgBoost.predict_proba(test[features])[:, 1]
watch['prediction'] = xgBoost.predict_proba(watch[features])[:, 1]


In [ ]:
results = pd.DataFrame(columns=['Metric', 'Train', 'Test', 'Watch'])

metrics = [
    ("Accuracy", accuracy_score),
    ("Precision", precision_score),
    ("Recall", recall_score),
    ("F1 Score", f1_score),
    ("AUC-ROC", roc_auc_score),
    ("Gini", gini),
    ("KS Statistic", ks_statistic),
    ("Jaccard", jaccard_score)
]

for metric_name, metric_func in metrics:
    if metric_name in ["Gini", "KS Statistic"]:  # Si la métrica requiere probabilidades
        train_score = metric_func(train['Default'], train['prediction'])
        test_score = metric_func(test['Default'], test['prediction'])
        watch_score = metric_func(watch['Default'], watch['prediction'])

    else:  # Si la métrica se aplica a etiquetas
        train_score = metric_func(train['Default'], train['prediction'].apply(lambda x: 1 if x > 0.5 else 0))
        test_score = metric_func(test['Default'],   test['prediction'].apply(lambda x: 1 if x > 0.5 else 0))
        watch_score = metric_func(watch['Default'], watch['prediction'].apply(lambda x: 1 if x > 0.5 else 0))

    results = results.append({
        'Metric': metric_name,
        'Train': train_score,
        'Test': test_score,
        'Watch': watch_score
    }, ignore_index=True)


pd.set_option('display.float_format', '{:.2f}'.format)

# Mostrar los resultados
results

### 8. Interpretación de modelo

In [88]:
%%capture
!pip install shap

In [ ]:
explainer = shap.Explainer(xgBoost,
                           train[features],
                           feature_names = features)

shap_values = explainer(train[features])

**PDP: Partial dependency plot**

Estos gráficos muestran cómo una característica afecta las predicciones manteniendo constantes las otras características.

In [ ]:
shap.plots.scatter(shap_values[:,"Score_Source_3"])

In [ ]:
shap.plots.scatter(shap_values[:,"Score_Source_1"], color = shap_values)

**Waterfall de SHAP**

Descompone la contribución de cada característica a una predicción individual, desde la predicción base (es decir, el valor promedio de todas las predicciones) hasta la predicción final para esa instancia específica.



In [ ]:
shap.plots.waterfall(shap_values[2])

**Beeswarm de SHAP**

El eje x muestra el valor SHAP, lo que indica cuánto influyó cada característica en la diferencia entre la predicción del modelo para esa instancia y la salida promedio del modelo en todo el conjunto de datos. Un valor SHAP positivo indica que esa característica hizo que la predicción aumentara, mientras que un valor SHAP negativo indica que la característica hizo que la predicción disminuyera.

In [ ]:
shap.plots.beeswarm(shap_values)

### 9. Implementación del modelo

In [121]:
import pickle

In [125]:
type(xgBoost)

xgboost.sklearn.XGBClassifier

Guardando objeto de Python

In [ ]:
pickle.dump(xgBoost, open("./XGBoost.xgb", 'wb'))
print("done")

Leyendo objeto de Python

In [ ]:
loaded_model = pickle.load(open("./XGBoost.xgb", 'rb'))

In [ ]:
loaded_model

Puntuando nuevas bases

In [ ]:
loaded_model.predict(xgb.DMatrix(X_test[features], label = y_test))

---
## Gracias por completar este laboratorio!